In [1]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import register_keras_serializable
from transformers import TFBertModel
import pandas as pd

@register_keras_serializable()
class BertLayer(Layer):
    def __init__(self, model_name="bert-base-uncased", **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.model_name = model_name  
        self.bert = TFBertModel.from_pretrained(model_name)  

    def call(self, inputs):
        return self.bert(inputs)[1]  

    def get_config(self):
        config = super(BertLayer, self).get_config()
        config.update({
            "model_name": self.model_name, 
        })
        return config

    @classmethod
    def from_config(cls, config):
        model_name = config.pop("model_name")
        return cls(model_name=model_name, **config)
########

In [2]:
import tensorflow as tf
model1 = tf.keras.models.load_model('mymodel_path.keras', custom_objects={'BertLayer': BertLayer, 'TFBertModel': TFBertModel})

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [3]:
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ attention_mask      │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_ids           │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_layer          │ (None, 768)       │          0 │ attention_mask[0… │
│ (BertLayer)         │                   │            │ input_ids[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 768)       │          0 │ bert_layer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     49,216 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 147,845 (577.52 KB)

 Trainable params: 49,281 (192.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 98,564 (385.02 KB)

In [4]:
from flask import Flask, request, jsonify, render_template
from transformers import BertTokenizer, TFBertModel
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
def tokenize(X):

    X = bert_tokenizer(
        text = list(X),
        add_special_tokens = True,
        max_length = 100,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True
        )
    return X

In [6]:
def pred(text):
    df = pd.DataFrame({'input1': [text]})
    text = tokenize(df)
    print(text['input_ids'].shape)
    values = model1.predict({
        'input_ids' : text['input_ids'],
        'attention_mask': text['attention_mask']
    })
    val = values[0][0]
    print(val)
    if(val > 0.5):
        return 1
    else:
        return 0

In [7]:
text = '''Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is DisturbingDonald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise abov
shdgbsdg'''
y = pred(text)
y

(1, 100)


C:\Users\suresh hanuman\AppData\Roaming\Python\Python311\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_ids', 'attention_mask']. Received: the structure of inputs={'input_ids': '*', 'attention_mask': '*'}
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
0.4891533


0

In [ ]:
flask_app = Flask(__name__)
@flask_app.route("/")
def Home():
    return render_template("index.html")

@flask_app.route("/predict", methods = ["POST"])
def predict():
    
    sl = str(request.form['Sepal_Length'])
    sw = str(request.form['Sepal_Width'])
    classes = ["True" , "Fake"]
    # prediction = classes[prediction]
    prediction = sl + sw
    val = classes[pred(prediction)]
    prediction = val
    return render_template("index.html", output = "{}".format(prediction))

if __name__ == "__main__":
    flask_app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Nov/2024 23:39:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2024 23:39:26] "GET / HTTP/1.1" 200 -
